In [1]:
import cv2
import os
from PIL import Image,ImageOps
import numpy as np
image_directory = 'Datasets/'

dataset = []
label = []
INPUT_SIZE = 64
no_tumor_brain = os.listdir(image_directory + 'no/')
for i, image_name in enumerate(no_tumor_brain):
    image = cv2.imread(image_directory + '/no/' + image_name)
    image = Image.fromarray(image,'RGB')
    max_side = max(image.size)
    # Tính toán padding
    left_padding = (max_side - image.width) // 2
    top_padding = (max_side - image.height) // 2
    right_padding = max_side - image.width - left_padding
    bottom_padding = max_side - image.height - top_padding

    # Thêm padding để tạo hình vuông
    image = ImageOps.expand(image, (left_padding, top_padding, right_padding, bottom_padding), fill=(0, 0, 0)) # Phần thêm sẽ để màu đen

    # Tùy chọn: Thay đổi kích thước hình ảnh thành hình vuông với kích thước mong muốn
    final_size = (INPUT_SIZE, INPUT_SIZE)
    image = image.resize(final_size)
    dataset.append(np.array(image))
    label.append(0)


yes_tumor_brain = os.listdir(image_directory + 'yes/')
for i, image_name in enumerate(yes_tumor_brain):
    image = cv2.imread(image_directory + '/yes/' + image_name)
    image = Image.fromarray(image,'RGB')
    max_side = max(image.size)
    # Tính toán padding
    left_padding = (max_side - image.width) // 2
    top_padding = (max_side - image.height) // 2
    right_padding = max_side - image.width - left_padding
    bottom_padding = max_side - image.height - top_padding

    # Thêm padding để tạo hình vuông
    image = ImageOps.expand(image, (left_padding, top_padding, right_padding, bottom_padding), fill=(0, 0, 0)) # Phần thêm sẽ để màu đen

    # Tùy chọn: Thay đổi kích thước hình ảnh thành hình vuông với kích thước mong muốn
    final_size = (INPUT_SIZE, INPUT_SIZE)
    image = image.resize(final_size)
    dataset.append(np.array(image))
    label.append(1)

print(dataset[0])
print(label[0])

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
0


In [2]:
dataset = np.array(dataset)
label = np.array(label)

In [3]:
from sklearn.model_selection import train_test_split
 

In [4]:
x_train, x_test, y_train, y_test = train_test_split(dataset,label, test_size=0.2, random_state=42)


In [5]:
from tensorflow import keras
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Activation,Dropout, Flatten, Dense

In [6]:
x_train = normalize(x_train,axis=1)
x_test = normalize(x_test,axis=1)


In [7]:
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))





f:\AI\myenvs\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.fit(x_train,y_train, 
        batch_size= 16,
        verbose=1,
        epochs=10,
        validation_data=(x_test,y_test),
        shuffle=False)

model.save("BrainTumor10epochs.h5")

Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.6350 - loss: 0.6344 - val_accuracy: 0.7533 - val_loss: 0.5003
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.7913 - loss: 0.4663 - val_accuracy: 0.8217 - val_loss: 0.3930
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8579 - loss: 0.3543 - val_accuracy: 0.8833 - val_loss: 0.2979
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.8921 - loss: 0.2703 - val_accuracy: 0.8817 - val_loss: 0.2933
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9212 - loss: 0.2053 - val_accuracy: 0.9333 - val_loss: 0.1840
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9497 - loss: 0.1403 - val_accuracy: 0.9500 - val_loss: 0.1339
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.9646 - loss: 0.0965 - val_accuracy: 0.9333 - val_loss: 0.1779
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.9759 - loss: 0.0699 - val_accu